<a href="https://colab.research.google.com/github/LeonardoR-Silva/Analise_ZOOP_Megastore/blob/main/Analise_ZOOP_Megastore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações


In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [79]:
raw_clientes = 'https://raw.githubusercontent.com/LeonardoR-Silva/Analise_ZOOP_Megastore/refs/heads/main/Dados/clientes(in).csv'
raw_vendas = 'https://raw.githubusercontent.com/LeonardoR-Silva/Analise_ZOOP_Megastore/refs/heads/main/Dados/vendas(in).csv'

dados_clientes = pd.read_csv(raw_clientes)
dados_vendas = pd.read_csv(raw_vendas)

##Verificando dados e tratando erros do dataframe

In [80]:
dados_clientes.head()

,id_cliente,nome_cliente,email,estado,plano_assinatura
0,101,Enzo Gomes,enzo.gomes@email.com,GO,Não
1,102,Heloísa Alves,heloísa.alves@email.com,SC,Não
2,103,Marcos Correia,marcos.correia@email.com,SC,Não
3,104,Luiza Silva,luiza.silva@email.com,SC,Não
4,105,Luiza Machado,luiza.machado@email.com,SC,Não


In [81]:
dados_clientes.shape

(50, 5)

In [82]:
dados_vendas.head()

,id_venda,data_venda,id_cliente,id_produto,produto,categoria,valor_venda,quantidade
0,1,2023-02-07,109.0,8.0,Livro: A Guerra dos Tronos,Livros,2677.12,2.0
1,2,2023-01-11,139.0,7.0,Headset Surround 7.1,Áudio,1210.10,1.0
2,3,2023-02-16,104.0,17.0,Mousepad Gamer XXL,Periféricos,2393.06,2.0
3,4,2023-03-20,145.0,17.0,Mousepad Gamer XXL,Periféricos,3925.92,3.0
4,5,2023-02-16,134.0,16.0,Memória RAM 16GB DDR5,Componentes,4702.89,1.0


In [83]:
dados_vendas.shape

(120, 8)

In [84]:
dados_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id_cliente        50 non-null     int64 
 1   nome_cliente      50 non-null     object
 2   email             50 non-null     object
 3   estado            50 non-null     object
 4   plano_assinatura  50 non-null     object
dtypes: int64(1), object(4)
memory usage: 2.1+ KB


In [85]:
dados_vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_venda     120 non-null    object 
 1   data_venda   112 non-null    object 
 2   id_cliente   112 non-null    float64
 3   id_produto   112 non-null    float64
 4   produto      112 non-null    object 
 5   categoria    112 non-null    object 
 6   valor_venda  112 non-null    float64
 7   quantidade   112 non-null    float64
dtypes: float64(4), object(4)
memory usage: 7.6+ KB


In [86]:
#Venda está como object
dados_vendas['data_venda'] = pd.to_datetime(dados_vendas['data_venda'])
dados_vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_venda     120 non-null    object        
 1   data_venda   112 non-null    datetime64[ns]
 2   id_cliente   112 non-null    float64       
 3   id_produto   112 non-null    float64       
 4   produto      112 non-null    object        
 5   categoria    112 non-null    object        
 6   valor_venda  112 non-null    float64       
 7   quantidade   112 non-null    float64       
dtypes: datetime64[ns](1), float64(4), object(3)
memory usage: 7.6+ KB


In [89]:
#vemos a presença de NaN no df de vendas
dados_vendas[dados_vendas.id_cliente.isnull()]

,id_venda,data_venda,id_cliente,id_produto,produto,categoria,valor_venda,quantidade
16,"17,2023-01-26,122,4,""Monitor Ultrawide 34"""""",P...",NaT,NaN,NaN,NaN,NaN,NaN,NaN
18,"19,2023-03-08,118,4,""Monitor Ultrawide 34"""""",P...",NaT,NaN,NaN,NaN,NaN,NaN,NaN
19,"20,2023-03-13,135,4,""Monitor Ultrawide 34"""""",P...",NaT,NaN,NaN,NaN,NaN,NaN,NaN
33,"34,2023-03-05,149,4,""Monitor Ultrawide 34"""""",P...",NaT,NaN,NaN,NaN,NaN,NaN,NaN
42,"43,2023-02-05,120,4,""Monitor Ultrawide 34"""""",P...",NaT,NaN,NaN,NaN,NaN,NaN,NaN
47,"48,2023-01-01,135,4,""Monitor Ultrawide 34"""""",P...",NaT,NaN,NaN,NaN,NaN,NaN,NaN
59,"60,2023-02-22,143,4,""Monitor Ultrawide 34"""""",P...",NaT,NaN,NaN,NaN,NaN,NaN,NaN
91,"92,2023-01-30,124,4,""Monitor Ultrawide 34"""""",P...",NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
pd.set_option('display.max_colwidth', 90)
dados_vendas[dados_vendas.id_cliente.isnull()].id_venda

,id_venda
16,"17,2023-01-26,122,4,""Monitor Ultrawide 34"""""",Periféricos,4952.4,3"
18,"19,2023-03-08,118,4,""Monitor Ultrawide 34"""""",Periféricos,2750.54,1"
19,"20,2023-03-13,135,4,""Monitor Ultrawide 34"""""",Periféricos,2230.8,2"
33,"34,2023-03-05,149,4,""Monitor Ultrawide 34"""""",Periféricos,4579.35,1"
42,"43,2023-02-05,120,4,""Monitor Ultrawide 34"""""",Periféricos,1732.21,1"
47,"48,2023-01-01,135,4,""Monitor Ultrawide 34"""""",Periféricos,3835.24,3"
59,"60,2023-02-22,143,4,""Monitor Ultrawide 34"""""",Periféricos,3904.65,4"
91,"92,2023-01-30,124,4,""Monitor Ultrawide 34"""""",Periféricos,208.2,3"


In [141]:
#Provavelmente ocorreu um erro ao incluir os dados das vendas com dados nulos, pois o dados das 8 vendas NaN estão como string
vendas_isnull = dados_vendas[dados_vendas.id_cliente.isnull()].id_venda
vendas_isnull.values[1]

'19,2023-03-08,118,4,"Monitor Ultrawide 34""",Periféricos,2750.54,1'

In [230]:
df_vendas_isnull_corrigido = pd.DataFrame([linha.split(',') for linha in vendas_isnull], columns = dados_vendas.columns)
df_vendas_isnull_corrigido

,id_venda,data_venda,id_cliente,id_produto,produto,categoria,valor_venda,quantidade
0,17,2023-01-26,122,4,"""Monitor Ultrawide 34""""""",Periféricos,4952.4,3
1,19,2023-03-08,118,4,"""Monitor Ultrawide 34""""""",Periféricos,2750.54,1
2,20,2023-03-13,135,4,"""Monitor Ultrawide 34""""""",Periféricos,2230.8,2
3,34,2023-03-05,149,4,"""Monitor Ultrawide 34""""""",Periféricos,4579.35,1
4,43,2023-02-05,120,4,"""Monitor Ultrawide 34""""""",Periféricos,1732.21,1
5,48,2023-01-01,135,4,"""Monitor Ultrawide 34""""""",Periféricos,3835.24,3
6,60,2023-02-22,143,4,"""Monitor Ultrawide 34""""""",Periféricos,3904.65,4
7,92,2023-01-30,124,4,"""Monitor Ultrawide 34""""""",Periféricos,208.2,3


In [233]:
df_vendas_isnull_corrigido['valor_venda'] = pd.to_numeric(df_vendas_isnull_corrigido['valor_venda'])
df_vendas_isnull_corrigido['quantidade'] = pd.to_numeric(df_vendas_isnull_corrigido['quantidade'])
df_vendas_isnull_corrigido['data_venda'] = pd.to_datetime(df_vendas_isnull_corrigido['data_venda'])

In [237]:
dados_vendas.dropna(inplace = True)
dados_vendas = pd.concat([dados_vendas,df_vendas_isnull_corrigido])
dados_vendas

,id_venda,data_venda,id_cliente,id_produto,produto,categoria,valor_venda,quantidade
0,1,2023-02-07,109.0,8.0,Livro: A Guerra dos Tronos,Livros,2677.12,2.0
1,2,2023-01-11,139.0,7.0,Headset Surround 7.1,Áudio,1210.10,1.0
2,3,2023-02-16,104.0,17.0,Mousepad Gamer XXL,Periféricos,2393.06,2.0
3,4,2023-03-20,145.0,17.0,Mousepad Gamer XXL,Periféricos,3925.92,3.0
4,5,2023-02-16,134.0,16.0,Memória RAM 16GB DDR5,Componentes,4702.89,1.0
...,...,...,...,...,...,...,...,...
3,34,2023-03-05,149,4,"""Monitor Ultrawide 34""""""",Periféricos,4579.35,1.0
4,43,2023-02-05,120,4,"""Monitor Ultrawide 34""""""",Periféricos,1732.21,1.0
5,48,2023-01-01,135,4,"""Monitor Ultrawide 34""""""",Periféricos,3835.24,3.0
6,60,2023-02-22,143,4,"""Monitor Ultrawide 34""""""",Periféricos,3904.65,4.0


In [238]:
dados_vendas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120 entries, 0 to 7
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_venda     120 non-null    object        
 1   data_venda   120 non-null    datetime64[ns]
 2   id_cliente   120 non-null    object        
 3   id_produto   120 non-null    object        
 4   produto      120 non-null    object        
 5   categoria    120 non-null    object        
 6   valor_venda  120 non-null    float64       
 7   quantidade   120 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 8.4+ KB
